In [1]:
from data_generator import *

In [2]:
test_cases = [
    "case30.m"
]

## Example of parsing active constraints: DC-OPF

In [3]:
org_dir = os.getcwd()
os.chdir('./matpower7.1/')

In [4]:
eng = matlab.engine.start_matlab()

In [5]:
test_case_idx = 0
case_name = test_cases[test_case_idx]

In [6]:
data = eng.dc_opf_solver(case_name, 0.03)

In [7]:
Pg = np.squeeze(np.array(data['Pg']))
B_idx = np.squeeze(np.array(data['B_idx']))
GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
F = np.squeeze(np.array(data['F']))
Pg_lim = np.array(data['Pg_lim'])
F_lim = np.array(data['F_lim'])
w = np.squeeze(np.array(data['w']))

### Generator active constraints: [Pg_min, Pg_max]]

In [8]:
Pg

array([44.07757128, 57.51722431, 22.10482288, 30.76156135, 15.26205725,
       15.2620573 ])

In [9]:
Pg_lim

array([[80.,  0.],
       [80.,  0.],
       [50.,  0.],
       [55.,  0.],
       [30.,  0.],
       [40.,  0.]])

In [10]:
Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
Pg_active

array([0., 0., 0., 0., 0., 0.])

### Bus active constraints, due to precense of co-related generation: [Pb_min, Pb_max]

In [11]:
Pb = get_Pb(Pg, GB_map)
Pb

{1: 44.077571281953965,
 2: 57.517224308359246,
 22: 22.10482288398564,
 27: 30.761561353321333,
 23: 15.262057250924663,
 13: 15.26205729615106}

In [12]:
Pb_lim = get_Pb_lim(Pg_lim, GB_map)
Pb_lim

{1: array([80.,  0.]),
 2: array([80.,  0.]),
 22: array([50.,  0.]),
 27: array([55.,  0.]),
 23: array([30.,  0.]),
 13: array([40.,  0.])}

In [13]:
Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
Pb_active

array([0., 0., 0., 0., 0., 0.])

### Flow Constraints: [-F_max, F_max] 

In [14]:
F

array([ 22.63295629,  21.44461499,  20.42222701,  18.88197802,
        15.35802146,  23.86379127,  20.59259589,  15.35802146,
         7.73855548,  24.37498635,   8.79138051,   5.023646  ,
         0.        ,   8.79138051,  11.30818466, -15.2620573 ,
         4.60580495,   6.74095875,   5.04321253,  -1.60592325,
         1.5216911 ,   6.45043684,   3.39344781,  -5.70497736,
         7.81144266,   7.51182788,  -2.8375209 ,  -3.98884108,
       -19.98498491,  -9.33957459,  -1.8690031 ,   2.79133758,
        -7.35217412,   3.39135723, -10.74353135,  -6.78883282,
         6.12788903,   7.10130815,   3.74904778,  -5.31665163,
        -1.47218119])

In [15]:
F_lim

array([[130.],
       [130.],
       [ 65.],
       [130.],
       [130.],
       [ 65.],
       [ 90.],
       [ 70.],
       [130.],
       [ 32.],
       [ 65.],
       [ 32.],
       [ 65.],
       [ 65.],
       [ 65.],
       [ 65.],
       [ 32.],
       [ 32.],
       [ 32.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 32.],
       [ 32.],
       [ 32.],
       [ 32.],
       [ 32.],
       [ 32.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 65.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 32.],
       [ 32.]])

In [16]:
F_active = get_F_active_constraints(F, F_lim)
F_active

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Merge all active constriants

In [17]:
active_aconstraints = merge_active_constraints(Pg_active, Pb_active, F_active)
active_aconstraints

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [19]:
eng.quit()

In [20]:
os.chdir(org_dir)

## Build Datasets for NNs

### example of creating a dataset
- **x**: uncertainty realization as a feature input
- **y**: active constraints index as a label output

In [21]:
dataset = create_dataset(case_name, dataset_size=1)
dataset

> creating dataset with case30.m
100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


{'x': array([ 0.        , -0.74674264, -0.07695867, -0.1845657 ,  0.        ,
         0.        ,  0.22243033, -0.67943549,  0.        , -0.29780386,
         0.        , -0.08112621,  0.        ,  0.0581917 , -0.21276046,
        -0.00315539, -0.04451734,  0.0602599 ,  0.31158072,  0.07321204,
        -0.45341773,  0.        , -0.11655524, -0.29062369,  0.        ,
         0.16092618,  0.        ,  0.        , -0.01624208,  0.35531925]),
 'y': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]])}

### Build datasets

In [24]:
# dataset_size = 2
dataset_size = 10

In [25]:
build_datasets(test_cases, dataset_size)

> creating dataset with case30.m
100%|██████████| 10/10 [00:04<00:00,  2.07it/s]<built-in function chdir>



### Check datasets

In [26]:
infile = open('./../../data/' + str(dataset_size) +'/case30.pickle','rb')
dataset = pickle.load(infile)
infile.close()

In [27]:
dataset

{'x': array([[ 0.        , -0.74674264, -0.07695867, -0.1845657 ,  0.        ,
          0.        ,  0.22243033, -0.67943549,  0.        , -0.29780386,
          0.        , -0.08112621,  0.        ,  0.0581917 , -0.21276046,
         -0.00315539, -0.04451734,  0.0602599 ,  0.31158072,  0.07321204,
         -0.45341773,  0.        , -0.11655524, -0.29062369,  0.        ,
          0.16092618,  0.        ,  0.        , -0.01624208,  0.35531925],
        [ 0.        ,  0.02119491,  0.03978195,  0.25093913,  0.        ,
          0.        , -1.02024777, -0.66807165,  0.        ,  0.40897956,
          0.        ,  0.2513538 ,  0.        ,  0.16528154, -0.18815291,
         -0.14723824, -0.3840415 ,  0.04686662, -0.05055192, -0.01293953,
          0.74513783,  0.        ,  0.01898986,  0.41438946,  0.        ,
          0.07314556,  0.        ,  0.        ,  0.01552825, -0.37073837],
        [ 0.        ,  0.06827344,  0.05200229,  0.58949201,  0.        ,
          0.        , -0.056426